In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [34]:
PROJECT_PATH = '../'
DATA_PATH = '/scratch/ceph/swei20/data/ae/dataset/test'
PCA_PATH = '/scratch/ceph/dobos/data/pfsspec/import/stellar/rbf/bosz_5000_full/pca/spectra.h5'
sys.path.insert(0, PROJECT_PATH)

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import tensorflow as tf
# tf.enable_v2_behavior()
gpus = tf.config.list_physical_devices('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Loading Data

In [35]:
with h5py.File(PCA_PATH, 'r') as f:
    flux = f['flux'][()]

In [39]:
def prepro_pca(flux, dim=300):
    flux0 = flux.reshape(-1, 1000)[:, :dim]    
    flux_clean = flux0[~np.isnan(flux0[:,0])]    
    print(flux_clean.shape, flux0.shape)
    print('min', flux_clean.min(), 'max', flux_clean.max(),'mean', flux_clean.mean(),'std', flux_clean.std() )
    return flux_clean

In [40]:
flux1 = prepro_pca(flux, dim=300)

(116614, 300) (243936, 300)
min -184.9254058126797 max 145.24091665337806 mean -0.10353327716635463 std 2.4000601615265493


In [155]:
SAVE_PATH = os.path.join(DATA_PATH, 'pca.h5')

In [156]:
with h5py.File(SAVE_PATH, 'w') as f:
    f.create_dataset('flux300', data = flux1)
#     f.create_dataset('flux10kmm', data = flux10kmm)

In [27]:
with h5py.File(fn, 'r') as f:
    flux10kmm = f['flux10kmm'][()]
#     flux10kmm = f['flux10kmm'][()]

## AE

In [125]:
from dotmap import DotMap

In [157]:
config_dict = {
    "data": {
      "dir": "/scratch/ceph/swei20/data/ae/dataset/test",
      "filename": "pca.h5",
      "train": "flux300",
      "test": None
    },
    "model":{
      "name": "SimpleAE",
      "input_dim": 300,
      "latent_dim": 8,
      "hidden_dims": [64],
      "dropout": 0.2,
      "lr": 0.001,
      "reg1": 0.0,
      "loss": "mae",
      "opt": "adam"
    },
    "trainer":{
      "epoch": 20,
      "batch_size": 32,
      "validation_split":0.1,
      "verbose_training": 1
    },
    "callbacks":{
        "checkpoint_monitor": "val_loss",
        "checkpoint_mode": "min"
    }
}

In [158]:
config = DotMap(config_dict)

In [163]:
from ae.data_loader.spec_data_loader import SpecDataLoader

In [164]:
ds = SpecDataLoader()

In [165]:
ds.init_from_config(config)

In [167]:
ds.get_train_data()[0].shape

(116614, 300)

In [159]:
from ae.model.simple_ae_model import SimpleAEModel

In [160]:
mm = SimpleAEModel()

In [161]:
mm.build_model(config)

[300, 64, 8]


In [162]:
from ae.trainer.simple_ae_trainer import SimpleAETrainer

In [153]:
tt = SimpleAETrainer(mm, flux1, config)

In [154]:
history = tt.train(ep=100)

Epoch 1/100
3280/3280 [==============================] - 8s 2ms/step - loss: 0.1552 - acc: 0.3957 - val_loss: 0.1679 - val_acc: 0.3021
Epoch 2/100
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1222 - acc: 0.5486 - val_loss: 0.1598 - val_acc: 0.3779
Epoch 3/100
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1196 - acc: 0.5719 - val_loss: 0.1610 - val_acc: 0.4273
Epoch 4/100
3280/3280 [==============================] - 6s 2ms/step - loss: 0.1178 - acc: 0.5896 - val_loss: 0.1527 - val_acc: 0.3950
Epoch 5/100
3280/3280 [==============================] - 6s 2ms/step - loss: 0.1164 - acc: 0.6084 - val_loss: 0.1553 - val_acc: 0.4733
Epoch 6/100
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1153 - acc: 0.6134 - val_loss: 0.1601 - val_acc: 0.4895
Epoch 7/100
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1147 - acc: 0.6171 - val_loss: 0.1590 - val_acc: 0.4863
Epoch 8/100
3280/3280 [==============================] 

In [136]:
history = tt.train()

Epoch 1/20
3280/3280 [==============================] - 8s 2ms/step - loss: 0.1587 - acc: 0.3878 - val_loss: 0.1785 - val_acc: 0.2356
Epoch 2/20
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1242 - acc: 0.5210 - val_loss: 0.1711 - val_acc: 0.3381
Epoch 3/20
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1210 - acc: 0.5409 - val_loss: 0.1697 - val_acc: 0.4019
Epoch 4/20
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1193 - acc: 0.5449 - val_loss: 0.1688 - val_acc: 0.3716
Epoch 5/20
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1182 - acc: 0.5559 - val_loss: 0.1669 - val_acc: 0.3912
Epoch 6/20
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1173 - acc: 0.5611 - val_loss: 0.1611 - val_acc: 0.4010
Epoch 7/20
3280/3280 [==============================] - 7s 2ms/step - loss: 0.1164 - acc: 0.5663 - val_loss: 0.1733 - val_acc: 0.4659
Epoch 8/20
3280/3280 [==============================] - 7s 2ms

In [138]:
root = "/home/swei20/AE/"

In [141]:
os.path.join(root, 'logs/fit/', mm.name)

'/home/swei20/AE/logs/fit/mae_lr3_l8_h2dp02_SimpleAE_0629_075839'

In [213]:
%%python 
--config "/home/swei20/AE/configs/ae/train/pca_config.json" --lr 2

  File "<stdin>", line 1
    --config "/home/swei20/AE/configs/ae/train/pca_config.json" --lr 2
             ^
SyntaxError: invalid syntax


CalledProcessError: Command 'b'--config "/home/swei20/AE/configs/ae/train/pca_config.json" --lr 2\n'' returned non-zero exit status 1.

In [ ]:
if __name__ == '__main__':
    print(sys.argv)

In [258]:
from ae.base.base_pipeline import BasePipeline

In [261]:
from ae.pipeline.simple_ae_pipeline import SimpleAEPipeline

In [259]:
cmd = "main --config /home/swei20/AE/configs/ae/train/pca_config.json --lr 2"
sys.argv = cmd.split()

In [263]:
sp=SimpleAEPipeline()
sp.prepare()
sp.run()

0.001
2


In [247]:
a = bp.args

In [255]:
a["pp"]

TypeError: 'int' object does not support item assignment

In [243]:
for key, value in a.item():
    print(key, value)

AttributeError: 'Namespace' object has no attribute 'item'

In [249]:
aa = DotMap(a)

In [251]:
aa.lr

In [224]:
%tb

SystemExit: 2